## 做一些准备工作

引入程序包，定义必要的全局变量。

### 引入库及模块

In [1]:
# 控件库
import ipywidgets as widgets
# 显示控件的方法
from IPython.display import display
# 时间日期
import datetime
# 数据计算包
import numpy as np
import pandas as pd
import statsmodels.api as sm

### 预定义变量

In [2]:
# @description 基于指定的 field 将 dict_list 转换为 list
# @param list 待处理的 dict_list
# @param aim_list? 目标输出 list
# @param field? 要提取的字段名
# @return list 一个新的 list
def dicts2list(dict_list, aim_list=[], field='value'):
    new_list = aim_list[:]
    for item in dict_list:
        new_list.append(item[field])
    return new_list


# @description
# @param dict 需要遍历处理的字典
# @param fn 字典字段值的处理函数，该函数第一参数为字典字段值，输出处理后的字段值
# @param fn_args 传给处理函数（fn）的参数
# @return dict 一个新的字典
def dictForEach(dict, fn, fn_args):
    new_dict = {}
    for name, item in dict.items():
        new_dict[name] = fn(item, **fn_args)
    return new_dict

In [3]:
# 预定义属性名列表
attr_list = tuple([{
    "zh": "颜值",
    "en": "look"
}, {
    "zh": "成绩",
    "en": "grade"
}, {
    "zh": "兴趣",
    "en": "hobby"
}, {
    "zh": "年龄",
    "en": "age"
},{
    "zh": "身高",
    "en": "height"
}, {
    "zh": "家庭",
    "en": "family"
}])
attr_list_en = tuple(dicts2list(attr_list, field = 'en'))
attr_list_zh = tuple(dicts2list(attr_list, field = 'zh'))

# 预定义属性及值的字典
dict = {
    "look": [{
        "name": "低",
        "value": 0
    }, {
        "name": "一般",
        "value": 1
    }, {
        "name": "高",
        "value": 2
    }],
    "grade": [{
        "name": "较差",
        "value": 0
    }, {
        "name": "一般",
        "value": 1 
    }, {
        "name": "优秀",
        "value": 2
    }],
    "hobby": [{
        "name": "毫不相关",
        "value": 0
    }, {
        "name": "互补",
        "value": 1
    }, {
        "name": "相似",
        "value": 2
    }],
    "age": [{
        "name": "更小",
        "value": 0
    }, {
        "name": "同级",
        "value": 1
    }, {
        "name": "更大",
        "value": 2
    }],
    "height": [{
        "name": "对方更矮一些",
        "value": 0
    }, {
        "name": "相似（5cm以内）",
        "value": 1
    }, {
        "name": "对方更高一些",
        "value": 2
    }],
    "family": [{
        "name": "不那么好",
        "value": 0
    }, {
        "name": "一般",
        "value": 1 
    }, {
        "name": "更好",
        "value": 2
    }]
}
# 预定义属性及值的列表
name_list_dict = dictForEach(dict, dicts2list, {"field": "name"})
value_list_dict = dictForEach(dict, dicts2list, {"field": "value"})

In [4]:
# print('预定义的属性包括：{keys}'.format(keys = dict.keys()))
# print('预定义属性及值的列表：{name_list_dict}'.format(name_list_dict = name_list_dict))

### 额外信息

In [5]:
# 容器
dropdowns = {}
# @description 根据预定义属性及其选项值设置widget，并初始化相关控制器
def generateDropdowns(cfg):
    global dropdowns
    for i in range(len(attr_list)):
        _item = dropdowns[attr_list_en[i]] = {}
        _item["initialed"] = False
        # dropdown 的选项
        _options = name_list_dict[attr_list_en[i]][:]
        _options.append(cfg["placeholder"])
        # dropdown 的描述
        _description = attr_list_zh[i]
        # 创建dropdown
        slt = widgets.Dropdown(options=_options,
                               description=_description,
                               description_tooltip='请选择…',
                               disabled=False)
        # 为 dropdown widget 实例绑定事件监听
        slt.observe(handleDropdownChange, names=['index'])
        slt.index = len(_options) - 1
        _item["widget"] = slt
        _item["options"] = _options
        _item["changed"] = False
        _item["initialed"] = True


def handleDropdownChange(detail):
    # 取到对应的英文属性名
    attr_name_zh = detail.owner.description
    attr_name_en = attr_list_en[list(attr_list_zh).index(attr_name_zh)]
    # 获取目标dropdown
    dropdown = dropdowns[attr_name_en]
    # 未初始化完成，什么都不做
    if not dropdown["initialed"]:
        return
    # 第一次更新值的时候将placeholder去掉
    if not dropdown["changed"]:
        dropdown["changed"] = True
        _index = detail["new"]
        dropdown["options"].pop()
        dropdown["widget"].options = dropdown["options"]
        dropdown["widget"].index = _index

# 生成并初始化 dropdowns
generateDropdowns({
    "placeholder": "点击进行选择…"
})

# 提取封装的widget,用于展示
dropdown_list = dicts2list(list(dropdowns.values()), field='widget')
display(*dropdown_list)

save_btn2 = widgets.Button(
    description='确认',
    disabled=False,
    button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='确认所有字段填答无误并保存',
    icon=''
)

values = {}
def save_values(detail):
    # 确认所有字段填写完毕
    changes = dicts2list(list(dropdowns.values()), field='changed')
    if changes.count(False) >= 1 :
        detail.button_style = "warning"
        return
    
    # 提取字段值
    for attr in attr_list_en:
        values[attr] = dropdowns[attr]["widget"].value
    
    # 更新按钮状态以示提交
    detail.icon = "check"
    detail.button_style = "success"
    detail.description = "保存成功"
    print(values)

save_btn2.on_click(save_values)
display(save_btn2)

Dropdown(description='颜值', description_tooltip='请选择…', index=3, options=('低', '一般', '高', '点击进行选择…'), value='点击…

Dropdown(description='成绩', description_tooltip='请选择…', index=3, options=('较差', '一般', '优秀', '点击进行选择…'), value='…

Dropdown(description='兴趣', description_tooltip='请选择…', index=3, options=('毫不相关', '互补', '相似', '点击进行选择…'), value…

Dropdown(description='年龄', description_tooltip='请选择…', index=3, options=('更小', '同级', '更大', '点击进行选择…'), value='…

Dropdown(description='身高', description_tooltip='请选择…', index=3, options=('对方更矮一些', '相似（5cm以内）', '对方更高一些', '点击进…

Dropdown(description='家庭', description_tooltip='请选择…', index=3, options=('不那么好', '一般', '更好', '点击进行选择…'), value…

Button(button_style='primary', description='确认', style=ButtonStyle(), tooltip='确认所有字段填答无误并保存')